In [ ]:
#Importing the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import category_encoders as ce

In [ ]:
 pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install category_encoders

In [ ]:
#Reading the overall train dataset
data = pd.read_csv('/content/train.csv')
#Displaying the first 5 rows of the overall train dataset
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [ ]:
#Getting the datatypes of the dataset
data.dtypes

,0
PassengerId,object
HomePlanet,object
CryoSleep,object
Cabin,object
Destination,object
Age,float64
VIP,object
RoomService,float64
FoodCourt,float64
ShoppingMall,float64


In [ ]:
#Consolidating the 3 columns by Total Expense
data_TotalExpenses = data[['RoomService', 'FoodCourt','ShoppingMall','Spa']]

#Calculate sum of each row and create 'sum' column
data_TotalExpenses['Total Expenses'] = data_TotalExpenses.sum(axis=1)

#Create 'Total Expenses' column in the original DataFrame
data['Total Expenses'] = data_TotalExpenses['Total Expenses']

data_updated = data[['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'Transported', 'Total Expenses']]

#Splitting the 'Cabin' column
data_updated[['Deck','Num', 'Side']] = data_updated['Cabin'].str.split('/', expand=True)

#Selecting the desired columns for the final data_updated DataFrame
data_updated = data_updated[['PassengerId', 'HomePlanet', 'CryoSleep', 'Deck', 'Num', 'Side', 'Destination', 'Age', 'VIP', 'Total Expenses','Transported']]
data_updated

<ipython-input-7-f438617072b6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_TotalExpenses['Total Expenses'] = data_TotalExpenses.sum(axis=1)
<ipython-input-7-f438617072b6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_updated[['Deck','Num', 'Side']] = data_updated['Cabin'].str.split('/', expand=True)
<ipython-input-7-f438617072b6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,PassengerId,HomePlanet,CryoSleep,Deck,Num,Side,Destination,Age,VIP,Total Expenses,Transported
0,0001_01,Europa,False,B,0,P,TRAPPIST-1e,39.0,False,0.0,False
1,0002_01,Earth,False,F,0,S,TRAPPIST-1e,24.0,False,692.0,True
2,0003_01,Europa,False,A,0,S,TRAPPIST-1e,58.0,True,10334.0,False
3,0003_02,Europa,False,A,0,S,TRAPPIST-1e,33.0,False,4983.0,False
4,0004_01,Earth,False,F,1,S,TRAPPIST-1e,16.0,False,1089.0,True
...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A,98,P,55 Cancri e,41.0,True,8462.0,False
8689,9278_01,Earth,True,G,1499,S,PSO J318.5-22,18.0,False,0.0,False
8690,9279_01,Earth,False,G,1500,S,TRAPPIST-1e,26.0,False,1873.0,True
8691,9280_01,Europa,False,E,608,S,55 Cancri e,32.0,False,1402.0,False


In [ ]:
#print the number of missing values in each column
print(data_updated.isnull().sum())

PassengerId         0
HomePlanet        201
CryoSleep         217
Deck              199
Num               199
Side              199
Destination       182
Age               179
VIP               203
Total Expenses      0
Transported         0
dtype: int64


***Using Mean for Handling missing values for numrical values : Age, Total Expenses***

In [ ]:
#Calculate the mean of the 'Age' column
mean_age = data_updated['Age'].mean()

# Fill missing values in 'Age' column with the mean
data_updated['Age'].fillna(mean_age, inplace=True)

# Calculate the mean of the 'Total Expenses' column
mean_total_expenses = data_updated['Total Expenses'].mean()

# Fill missing values in 'Total Expenses' column with the mean
data_updated['Total Expenses'].fillna(mean_total_expenses, inplace=True)

# Print the number of missing values in each column to verify
print(data_updated.isnull().sum())

PassengerId         0
HomePlanet        201
CryoSleep         217
Deck              199
Num               199
Side              199
Destination       182
Age                 0
VIP               203
Total Expenses      0
Transported         0
dtype: int64


In [ ]:
from sklearn.impute import KNNImputer

# Create a copy of the DataFrame to avoid modifying the original
data_imputed = data_updated.copy()

# Select relevant columns for KNN imputation (numerical and categorical)
features = ['Age', 'Total Expenses', 'HomePlanet']
data_subset = data_imputed[features]

# Convert categorical features to numerical using one-hot encoding
data_encoded = pd.get_dummies(data_subset, columns=['HomePlanet'])

# Initialize KNNImputer
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# Fit and transform the data
data_imputed_array = imputer.fit_transform(data_encoded)

# Convert the imputed array back to a DataFrame
data_imputed = pd.DataFrame(data_imputed_array, columns=data_encoded.columns)

# Reverse one-hot encoding to get 'HomePlanet' back
data_imputed['HomePlanet'] = data_imputed[['HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars']].idxmax(axis=1).str.replace('HomePlanet_', '')

# Drop the one-hot encoded columns
data_imputed = data_imputed.drop(['HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars'], axis=1)

# Update the original DataFrame with imputed 'HomePlanet' values
data_updated['HomePlanet'] = data_imputed['HomePlanet']

print(data_updated.isnull().sum())

PassengerId         0
HomePlanet          0
CryoSleep         217
Deck              199
Num               199
Side              199
Destination       182
Age                 0
VIP               203
Total Expenses      0
Transported         0
dtype: int64


***Using KNN for handling missing values in categorical data columns***

In [ ]:
# Create a copy of the DataFrame to avoid modifying the original
data_imputed = data_updated.copy()

# Select relevant columns for KNN imputation (numerical and categorical)
features = ['Age', 'Total Expenses', 'CryoSleep', 'Deck', 'Num', 'Side', 'Destination', 'VIP']
data_subset = data_imputed[features]

# Convert categorical features to numerical using one-hot encoding
data_encoded = pd.get_dummies(data_subset, columns=['CryoSleep', 'Deck', 'Side', 'Destination', 'VIP'], dummy_na=False) #dummy_na=False will ignore nan values during one-hot encoding

# Initialize KNNImputer
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# Fit and transform the data
data_imputed_array = imputer.fit_transform(data_encoded)

# Convert the imputed array back to a DataFrame
data_imputed = pd.DataFrame(data_imputed_array, columns=data_encoded.columns)

# Reverse one-hot encoding to get categorical features back
categorical_features = ['CryoSleep', 'Deck', 'Side', 'Destination', 'VIP']
for feature in categorical_features:
    #Get a list of columns related to the feature
    feature_cols = [col for col in data_imputed.columns if feature in col]

    #Find the column with the maximum value for each row and extract the category name
    data_imputed[feature] = data_imputed[feature_cols].idxmax(axis=1).str.replace(f'{feature}_', '')

    #Remove the one-hot encoded columns for the current feature
    data_imputed = data_imputed.drop(feature_cols, axis=1)

# Update the original DataFrame with imputed values
for feature in features:
    data_updated[feature] = data_imputed[feature]

print(data_updated.isnull().sum())

PassengerId       0
HomePlanet        0
CryoSleep         0
Deck              0
Num               0
Side              0
Destination       0
Age               0
VIP               0
Total Expenses    0
Transported       0
dtype: int64


In [ ]:
print(data_updated)

     PassengerId HomePlanet CryoSleep Deck     Num Side    Destination   Age  \
0        0001_01     Europa     False    B     0.0    P    TRAPPIST-1e  39.0   
1        0002_01      Earth     False    F     0.0    S    TRAPPIST-1e  24.0   
2        0003_01     Europa     False    A     0.0    S    TRAPPIST-1e  58.0   
3        0003_02     Europa     False    A     0.0    S    TRAPPIST-1e  33.0   
4        0004_01      Earth     False    F     1.0    S    TRAPPIST-1e  16.0   
...          ...        ...       ...  ...     ...  ...            ...   ...   
8688     9276_01     Europa     False    A    98.0    P    55 Cancri e  41.0   
8689     9278_01      Earth      True    G  1499.0    S  PSO J318.5-22  18.0   
8690     9279_01      Earth     False    G  1500.0    S    TRAPPIST-1e  26.0   
8691     9280_01     Europa     False    E   608.0    S    55 Cancri e  32.0   
8692     9280_02     Europa     False    E   608.0    S    TRAPPIST-1e  44.0   

        VIP  Total Expenses  Transporte

In [ ]:

data_imputed = data_updated.copy()

# Select relevant columns for KNN imputation (numerical and categorical)
features = ['Age', 'Total Expenses', 'CryoSleep', 'Deck', 'Num', 'Side', 'Destination', 'VIP']
data_subset = data_imputed[features]

# Convert categorical features to numerical using one-hot encoding
data_encoded = pd.get_dummies(data_subset, columns=['CryoSleep', 'Deck', 'Side', 'Destination', 'VIP'], dummy_na=False)

# Initialize KNNImputer
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# Fit and transform the data
data_imputed_array = imputer.fit_transform(data_encoded)

# Convert the imputed array back to a DataFrame
data_imputed = pd.DataFrame(data_imputed_array, columns=data_encoded.columns)

In [ ]:
# Reverse one-hot encoding to get categorical features back
categorical_features = ['CryoSleep', 'Deck', 'Side', 'Destination', 'VIP']
for feature in categorical_features:
    feature_cols = [col for col in data_imputed.columns if feature in col]
    data_imputed[feature] = data_imputed[feature_cols].idxmax(axis=1).str.replace(f'{feature}_', '')
    data_imputed = data_imputed.drop(feature_cols, axis=1)

# Update the original DataFrame with imputed values
for feature in features:
    data_updated[feature] = data_imputed[feature]

# Prepare features and target variable for CatBoost using data_updated
X = data_updated.drop(columns=['PassengerId', 'Transported'])
y = data_updated['Transported'].astype(int)  # Convert to integer (0 or 1)

# Define categorical columns for CatBoost
categorical_cols = ['CryoSleep', 'Deck', 'Side', 'Destination', 'VIP']


In [ ]:
# Ensure categorical columns are properly marked as 'category' dtype
for col in categorical_cols:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

# Confirm all numeric columns are of numeric dtype
# Use apply(pd.to_numeric, errors='coerce') to handle non-numeric columns gracefully
for col in X_train.columns:
    if col not in categorical_cols:
        X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
        X_test[col] = pd.to_numeric(X_test[col], errors='coerce')

# Ensure no numeric column contains text-like entries
print(X_train.dtypes)  # Check the data types
print(X_test.dtypes)

# Initialize CatBoostClassifier model
model = CatBoostClassifier(verbose=100)

# Fit the model, explicitly specifying categorical features
model.fit(X_train, y_train, cat_features=categorical_cols)

# Make predictions on the test data
y_pred = model.predict(X_test)


HomePlanet         float64
CryoSleep         category
Deck              category
Num                float64
Side              category
Destination       category
Age                float64
VIP               category
Total Expenses     float64
dtype: object
HomePlanet         float64
CryoSleep         category
Deck              category
Num                float64
Side              category
Destination       category
Age                float64
VIP               category
Total Expenses     float64
dtype: object
Learning rate set to 0.023581
0:	learn: 0.6863641	total: 9.46ms	remaining: 9.45s
100:	learn: 0.5018077	total: 868ms	remaining: 7.72s
200:	learn: 0.4827167	total: 2.44s	remaining: 9.7s
300:	learn: 0.4716200	total: 4.14s	remaining: 9.63s
400:	learn: 0.4617766	total: 5.11s	remaining: 7.64s
500:	learn: 0.4500015	total: 6.02s	remaining: 5.99s
600:	learn: 0.4406323	total: 6.92s	remaining: 4.59s
700:	learn: 0.4321701	total: 7.81s	remaining: 3.33s
800:	learn: 0.4241136	total: 8.72s	remaini

**Using F1 score and AUC-ROC for model performance measure**

In [ ]:
# Make predictions on the test data
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

Accuracy: 0.7527
F1 Score: 0.7422
AUC-ROC: 0.8271
